# RF Chain and Channelisation Commissioning Test

## Table of Contents\n
#### [0.1 - Global Configuration](#02---setup-test-equipment)
#### [0.2 - Setup Test Equipment](#02---setup-test-equipment)
#### [1 -> Test Steps](#1---test-steps)
this notebook provides for indepenent control of the test equipment during the rf chain and channelisation test so that test equipment setup may be changed while the telescope is scanning.  All test step requirements are indicated but only test equipment steps are included in this notebook.

## 0.1 Global Configuration

### 0.1.1 Import dependencies

In [1]:
import sys

sys.path.append("../../src")

import json
import logging
import os
import pathlib
import time
from typing import List

import ska_ser_logging
from bokeh.io import output_notebook
from ska_oso_pdm.entities.common.target import (
    CrossScanParameters,
    FivePointParameters,
    RasterParameters,
    SinglePointParameters,
    StarRasterParameters,
)
from ska_oso_pdm.entities.sdp import BeamMapping
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions.devicecontrol.resource_control import get_request_json
from ska_oso_scripting.objects import SubArray, Telescope
from ska_tmc_cdm.messages.central_node.assign_resources import AssignResourcesRequest
from ska_tmc_cdm.messages.central_node.sdp import Channel
from ska_tmc_cdm.messages.subarray_node.configure import ConfigureRequest
from ska_tmc_cdm.messages.subarray_node.configure.core import ReceiverBand

from ska_mid_jupyter_notebooks.cluster.cluster import Environment, TangoDeployment
from ska_mid_jupyter_notebooks.dish.dish import TangoDishDeployment
from ska_mid_jupyter_notebooks.helpers.path import project_root
from ska_mid_jupyter_notebooks.obsconfig.config import ObservationSB
from ska_mid_jupyter_notebooks.obsconfig.target_spec import TargetSpec, get_default_target_specs_sb
from ska_mid_jupyter_notebooks.sut.rendering import TelescopeMononitorPlot
from ska_mid_jupyter_notebooks.sut.state import TelescopeDeviceModel, get_telescope_state
from ska_mid_jupyter_notebooks.sut.sut import TangoSUTDeployment, disable_qa
from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment

### 0.1.2 Setup Global Variables and Configuration

In [2]:
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured: {test_equipment}")
timestr = time.strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(), f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}"
)
os.makedirs(notebook_output_dir, exist_ok=True)
# we disable qa as it is not been properly verified
disable_qa()

Test Equipment Configured: TangoTestEquipment{namespace=test-equipment; tango_host=tango-databaseds.test-equipment.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.test-equipment.svc.miditf.internal.skao.int:8765}


### 0.1.3 Test Connections to Namespaces

In [4]:
test_equipment.smoke_test()

CIA PingResponse (test-equipment): {"result":"ok","time":"2024-05-16T07:23:59.417793"}
mid-itf/siggen/1 is reachable
mid-itf/progattenuator/1 is reachable
mid-itf/spectana/1 is reachable
mid-itf/skysimctl/4 is reachable


## 0.2 Test Equipment Setup

Use the noise source at nominal levels for input to at least one SPFRx.  

### 0.2.1 Configure Test Equipment State 

In [5]:
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices

['mid-itf/progattenuator/1',
 'mid-itf/siggen/1',
 'mid-itf/skysimctl/4',
 'mid-itf/spectana/1']

### 0.2.2 Print Test Equipment Diagnostics

In [6]:
test_equipment.print_diagnostics()

mid-itf/skysimctl/4 State: ON
mid-itf/skysimctl/4 Band: 1
mid-itf/skysimctl/4 Correlated_Noise_Source: True
mid-itf/skysimctl/4 Uncorrelated_Noise_Sources: False
mid-itf/skysimctl/4 H_Channel: False
mid-itf/skysimctl/4 V_Channel: False
mid-itf/skysimctl/4 temperature: 23.6
mid-itf/skysimctl/4 humidity: 0.0
mid-itf/siggen/1 versionId: 0.9.1
mid-itf/siggen/1 adminMode: 0
mid-itf/siggen/1 State: UNKNOWN
mid-itf/siggen/1 healthState: HealthState.UNKNOWN
mid-itf/siggen/1 frequency: 470000000.0
mid-itf/siggen/1 power_cycled: False
mid-itf/siggen/1 power_dbm: -25.0
mid-itf/siggen/1 rf_output_on: True
mid-itf/siggen/1 controlMode: 0
mid-itf/siggen/1 simulationMode: 0
mid-itf/siggen/1 testMode: 0
mid-itf/siggen/1 loggingLevel: 4
mid-itf/siggen/1 command_error: False
mid-itf/siggen/1 device_error: False
mid-itf/siggen/1 execution_error: False
mid-itf/siggen/1 query_error: False
mid-itf/progattenuator/1 versionId: 0.9.1
mid-itf/progattenuator/1 model_name: MN=RCDAT-8000-30
mid-itf/progattenuator/

### 0.2.3 Create Test Equipment Plot

In [7]:
monitor_plot = get_test_equipment_monitor_plot()
test_equipment_state.subscribe_to_test_equipment_state(monitor_plot.handle_device_state_change)
output_notebook()
monitor_plot.show()
test_equipment_state.activate()

Loading BokehJS ...

1|2024-05-16T15:23:38.474Z|WARNING|Thread-6 (_listening_daemon)|_listening_daemon|statemonitoring.py#1138||exiting monitoring loop due to an unknown exception


Exception in thread Thread-6 (_listening_daemon):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/daveh/ska-mid-jupyter-notebooks/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/daveh/ska-mid-jupyter-notebooks/src/ska_mid_jupyter_notebooks/monitoring/statemonitoring.py", line 1139, in _listening_daemon
    raise exception
  File "/home/daveh/ska-mid-jupyter-notebooks/src/ska_mid_jupyter_notebooks/monitoring/statemonitoring.py", line 1110, in _listening_daemon
    event_key_input = event.key
  File "/home/daveh/ska-mid-jupyter-notebooks/src/ska_mid_jupyter_notebooks/monitoring/statemonitoring.py", line 75, in key
    return event_key(self.device.name(), self.attr_name)
PyTango.CommunicationFailed: DevFailed[
DevEr

### 0.2.4 Turn offline Test Equipment devices ONLINE

In [8]:
# set any offline devices to online
test_equipment.turn_online()

set mid-itf/siggen/1 adminMode already ONLINE
set mid-itf/progattenuator/1 adminMode already ONLINE
set mid-itf/spectana/1 adminMode already ONLINE


### 0.2.5 Display Test Equipment Device States 

In [9]:
test_equipment_state.state["devices_states"]

{'mid-itf/progattenuator/1:state': 'ON',
 'mid-itf/siggen/1:state': 'UNKNOWN',
 'mid-itf/skysimctl/4:state': 'ON',
 'mid-itf/spectana/1:state': 'ON'}

**Step 1:**

Set CW generator to approximately 800MHz (freq1) & -30dBm (power1), and wideband noise to approximately -132dBm/Hz at the SPF Band 1 Receiver inputs. ie. Band 1 output on and attenuation approx 10dB.  

Expected Result:  
*Test equipment configured*

In [ ]:
#test equipment setup values - band 1
frequency_to_set = 800.0002e6
power_level = -30.0
attenuation = 10
band = 1

signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

#print current 
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")   

#setup siggen
signal_generator.write_attribute('frequency', frequency_to_set)
signal_generator.write_attribute('power_dbm', power_level)
#setup SSC
SSC.write_attribute('Correlated_Noise_Source', True)
SSC.write_attribute('Uncorrelated_Noise_Sources', True)
SSC.write_attribute('Band', band)
#setup Attenuator
prog_atten.write_attribute('channel_1', attenuation)

time.sleep(3)
#print updated values and confirm updates
print(f"Updated signal generator frequency                  : {signal_generator.frequency}")
print(f"Updated signal generator power level                : {signal_generator.power_dbm}")
print(f"Updated Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Updated Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Updated Sky Simulator Band                          : {SSC.Band}")
print(f"Updated Programmable attenuator current attenuation : {prog_atten.channel_1}") 

assert (
    signal_generator.frequency == frequency_to_set
), print(f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}")
assert (
    signal_generator.power_dbm == power_level
), print(f"Power level required is {power_level} but got {signal_generator.power_dbm}")
assert (
    SSC.Correlated_Noise_Source == True
), print(f"Correlated noise source required is {power_level} but got {SSC.Correlated_Noise_Source}")
assert (
    SSC.Uncorrelated_Noise_Sources == False
), print(f"Uncorrelated noise source required is {power_level} but got {SSC.Uncorrelated_Noise_Sources}")
assert (
    SSC.Band == band
), print(f"Band required is {band} but got {SSC.Band}")
assert (
    prog_atten.channel_1 == attenuation
), print(f"Attenuation required is {attenuation} but got {prog_atten.channel_1}")

In [10]:
#print current only
signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

#print current 
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")   

Current signal generator frequency: 470000000.0
Current signal generator power level: -25.0
Current Sky Simulator Correlated Noise Source: True
Current Sky Simulator Uncorrelated Noise Source: False
Current Sky Simulator Band: 1
Current Programmable attenuator current attenuation : 10.0


**Step 2:**

Assign resources, configure the system for a subarray to run an observation scan in Band 1, with correlator averaging set to no more than 2 seconds.  

Expected Result:  
*SUT Configured*

This section runs steps 3.1 to 3.8 of the Configure Scan Tests

**Step 3:**

Start scan and signal path recording and monitor the SDP Spectrometer Display.  

Expected Result:  
_CW + noise in SDP Spectrometer Display_
 
This section runs step 3.10 of the Configure Scan Tests and provides a link to the SDP signal displays

**Step 4:**

~10 seconds after recording has started, use Test Control Script to adjust CW generator signal level by -3dB. 

Expected Result:  
*CW level changes by roughly -3dB in SDP Spectrometer Display*

In [ ]:
time.sleep(10)     # cell to be run in this notebook immediately after starting scn in rf-chain-and-channelisation_test_equipment notebook
power_level_minus_3_db = power_level - 3.0
signal_generator = test_equipment.signal_generator
print(f"Current signal generator power level: {signal_generator.power_dbm}")
signal_generator.write_attribute('power_dbm', power_level_minus_3_db)
time.sleep(3)
print(f"Updated signal generator power level: {signal_generator.power_dbm}")

**Step 5:**

After a further ~10 seconds, use Test Control Script to adjust CW generator signal level by -3dB.  

Expected Result:  
*CW level changes by roughly -3dB in SDP Spectrometer Display*

In [ ]:
time.sleep(10)     # cell to be run in this notebook immediately after starting scn in rf-chain-and-channelisation_test_equipment notebook
power_level_minus_6_db = power_level - 6.0
print(f"Current signal generator power level: {signal_generator.power_dbm}")
signal_generator.write_attribute('power_dbm', power_level_minus_6_db)
time.sleep(3)
print(f"Updated signal generator power level: {signal_generator.power_dbm}")

**Step 6:**

After a further ~10seconds, use Test Control Script to increase the CW generator signal frequency by 10MHz.  

Expected Result:  
*CW frequency changes by 10MHz in SDP Spectrometer Display*

In [ ]:
time.sleep(10)
frequency_plus_10MHz = frequency_to_set + 10.0e6
print(f"Current signal generator frequency: {signal_generator.frequency}")
signal_generator.write_attribute('frequency', frequency_plus_10MHz)
time.sleep(3)
print(f"Updated signal generator frequency: {signal_generator.frequency}")

**Step 7:**

AAfter a further ~10seconds, end scan and recording. Monitor feedback from SDP storage. 

Expected Result:  
*SDP storage listing includes new dataset*

**Step 8:**

Retrieve the dataset from the SDP storage (using Data Product Dashboard).  

Expected Result:  
_Copy of data file successfully downloaded_

**Step 9:** 

ANALYSIS:  

Analysis steps not done in Notebook

**Step 10:**

Determine the receiver band of the dataset.  

Expected Result:  
_Band 1_

**Step 11:**

Determine the number of frequency channels recorded.  

Expected Result:  
_Number of frequency channels in the range [51180_

**Step 12:**

Process the data file to identify the dominant CW signal at each time interval. Extract the detected frequency and power level as a time series.  

Expected Result:  
_Record of CW frequency & power for each time interval_

**Step 13:**

Confirm that the time series agrees with the programmed sequence of attenuation and frequency offset steps.
Note: this test is not intended to verify accuracy of frequency & power, so rough agreement is adequate.

Expected Result:  
*1st ~10sec segment: freq1, power1*

*2nd ~10sec segment: freq1, power2*

*3rd ~10sec segment: freq1, power3*

*4th ~10sec segment: freq1+10MHz, power3*

**Step 14:**

  

Expected Result:  
__

**Step 15:**

Set CW generator to approximately 1500MHz (freq2) & -30dBm, and wideband noise to approximately -132dBm/Hz at the SPF Band 2 Receiver inputs.  ie. Band 2 output on and attenuation approx 10dB.

Expected Result:  
*Test equipment configured*

In [ ]:
#test equipment setup values - band 1
frequency_to_set = 1500.0002e6
power_level = -30.0
attenuation = 10
band = 2

signal_generator = test_equipment.signal_generator
SSC = test_equipment.sky_simulator_controller
prog_atten = test_equipment.programmable_attenuator

#print current 
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
print(f"Current Sky Simulator Correlated Noise Source: {SSC.Correlated_Noise_Source}")
print(f"Current Sky Simulator Uncorrelated Noise Source: {SSC.Uncorrelated_Noise_Sources}")
print(f"Current Sky Simulator Band: {SSC.Band}")
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")   

#setup siggen
signal_generator.write_attribute('frequency', frequency_to_set)
signal_generator.write_attribute('power_dbm', power_level)
#setup SSC
SSC.write_attribute('Correlated_Noise_Source', True)
SSC.write_attribute('Uncorrelated_Noise_Sources', False)
SSC.write_attribute('Band', band)
#setup Attenuator
prog_atten.write_attribute('channel_1', attenuation)

time.sleep(3)
#print updated values and confirm updates
print(f"Updated signal generator frequency                  : {signal_generator.frequency}")
print(f"Updated signal generator power level                : {signal_generator.power_dbm}")
print(f"Updated Sky Simulator Correlated Noise Source       : {SSC.Correlated_Noise_Source}")
print(f"Updated Sky Simulator Uncorrelated Noise Source     : {SSC.Uncorrelated_Noise_Sources}")
print(f"Updated Sky Simulator Band                          : {SSC.Band}")
print(f"Updated Programmable attenuator current attenuation : {prog_atten.channel_1}") 

assert (
    signal_generator.frequency == frequency_to_set
), print(f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}")
assert (
    signal_generator.power_dbm == power_level
), print(f"Power level required is {power_level} but got {signal_generator.power_dbm}")
assert (
    SSC.Correlated_Noise_Source == True
), print(f"Correlated noise source required is {power_level} but got {SSC.Correlated_Noise_Source}")
assert (
    SSC.Uncorrelated_Noise_Sources == False
), print(f"Uncorrelated noise source required is {power_level} but got {SSC.Uncorrelated_Noise_Sources}")
assert (
    SSC.Band == band
), print(f"Band required is {band} but got {SSC.Band}")
assert (
    prog_atten.channel_1 == attenuation
), print(f"Attenuation required is {attenuation} but got {prog_atten.channel_1}")
signal_generator = test_equipment.signal_generator
print(f"Current signal generator frequency: {signal_generator.frequency}")
print(f"Current signal generator power level: {signal_generator.power_dbm}")
signal_generator.write_attribute('frequency', frequency_to_set)
signal_generator.write_attribute('power_dbm', power_level)
time.sleep(3)
print(f"Updated signal generator frequency: {signal_generator.frequency}")
print(f"Updated signal generator power level: {signal_generator.power_dbm}")
assert (
    signal_generator.frequency == frequency_to_set
), print(f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}")
prog_atten = test_equipment.programmable_attenuator
print(prog_atten.get_attribute_list())
print(f"Current Programmable attenuator current attenuation : {prog_atten.channel_1}")   #print current attenuationprog_atten.write_attribute('channel_1', 10)

prog_atten.write_attribute('channel_1', attenuation)
time.sleep(1)
print(f"Updated Programmable attenuator attenuation : {prog_atten.channel_1}")   #print updated attenuation"frequency_to_set = 800e6

**Step 16:**

Assign resources, configure the system for a subarray to run an observation scan in Band 2, with correlator averaging set to no more than 2 seconds.

Expected Result:  
SUT configured

**Step 17:**

Start scan and start signal path recording and monitor the Spectrometer Display.  

Expected Result:  
_CW + noise in SDP Spectrometer Display_

**Step 18:**

After ~10seconds, end scan and recording. Monitor feedback from SDP storage .

Expected Result:  
*SDP storage listing includes new dataset*

**Step 19:**

Retrieve the dataset from the SDP storage (using Data Product Dashboard).  

Expected Result:  
_Copy of data file successfully downloaded_

**Step 20:**

ANALYSIS:  

Expected Result:  
__

**Step 21:**

Determine the number of frequency channels recorded.  

Expected Result:  
_Number of frequency channels in the range [51180, 65536] (inclusive). Note that only approximately 16000 channels are expected to be available prior to AA3_

**Step 22:**

Determine the receiver band of the dataset.  

Expected Result:  
_Band 2_

**Step 23:**

Process the data file to identify the dominant CW signal at each time interval. Extract the detected frequency and power level as a time series.  

Expected Result:  
_Record of CW frequency & power for each time interval_

**Step 24:**

Confirm that the dominant CW signal frequency agrees with the programmed signal frequency.

Note: this test is not intended to verify accuracy of frequency & power, so rough agreement is adequate. 

Expected Result:  
_freq2_